1. データセット wine に対して， 最初の行の例題を評価用， 残りの行の例題を訓練用として， 最近傍識別を行え． ただし，k-最近傍識別器 の近傍数は k=1 とせよ．

In [55]:
import pandas as pd
from sklearn import neighbors

data=pd.read_csv("yXT_wine.csv", header=None)
y=data.pop(0).values

y_test=y[0]
y_train=y[1:]

data_test=data.loc[0]
data_train=data.loc[1::]

knn=neighbors.KNeighborsClassifier(n_neighbors=1)
knn.fit(data_train, y_train)
pre=knn.predict([data_test])

2. 各データセットに対して， 最初の行の例題を評価用， 残りの行の例題を訓練用として， 最近傍識別を行え． ただし，k-最近傍識別器 の近傍数は k=1 とせよ．

In [77]:
import pandas as pd
import numpy as np
from sklearn import neighbors

dataset=['yXT_car.csv','yXT_seeds.csv','yXT_yeast.csv','yXT_winequality-white.csv']
pre=[]
for i in range(len(dataset)):
    data=pd.read_csv(dataset[i],header=None)
    y=data.pop(0).values
    y_test=y[0]
    y_train=y[1:]

    data_test=data.loc[0]
    data_train=data.loc[1::]

    knn=neighbors.KNeighborsClassifier(n_neighbors=1)
    knn.fit(data_train, y_train)
    pre=knn.predict([data_test])
    #print(pre==y_test)


3. データセット wine に対して， 各クラスのデータを訓練用と評価用を 7:3 にランダムに分けよ．

In [76]:
from sklearn.model_selection import train_test_split

data=pd.read_csv("yXT_wine.csv", header=None)
xtrain, xtest, ytrain, ytest=train_test_split(data, data[0], test_size=0.3, stratify=data[0],random_state=1)


4. データセット wine に対して， k-最近傍識別器 の k=1 とし，評価用データの識別率を出せ．

In [89]:
knn=neighbors.KNeighborsClassifier(n_neighbors=1)
knn.fit(xtrain, ytrain)
knn.score(xtest,ytest)

0.7592592592592593

5. step-3を１０回繰り返して，識別率の平均値と標準偏差を算出せよ．

In [104]:
data=pd.read_csv("yXT_wine.csv", header=None)
knn=neighbors.KNeighborsClassifier(n_neighbors=1)
score=[]
for i in range(10):
    xtrain,xtest,ytrain,ytest=train_test_split(data, data[0], test_size=0.3, stratify=data[0])
    knn.fit(xtrain,ytrain)
    score.append(knn.score(xtest, ytest))
print(np.mean(score))
print(np.std(score))

0.7537037037037038
0.026254531256958938


6. 各データセットに対して，各クラスのデータを訓練用と評価用を 7:3 にランダムに分け，k-最近傍識別器 の k=1 とし， 評価用データの識別率を出せ．

In [103]:
dataset=['yXT_car.csv','yXT_seeds.csv','yXT_yeast.csv','yXT_winequality-white.csv']
sco_car=[]
sco_seeds=[]
sco_yeast=[]
sco_winequality=[]
score=[sco_car,sco_seeds,sco_yeast,sco_winequality]
for i in range(len(dataset)):
    data=pd.read_csv(dataset[i],header=None)
    xtrain,xtest,ytrain,ytest=train_test_split(data, data[0], test_size=0.3, stratify=data[0])
    
    knn=neighbors.KNeighborsClassifier(n_neighbors=1)
    knn.fit(xtrain, ytrain)
    score[i].append(knn.score(xtest,ytest))

[0.636734693877551]

7. 各データセットに対して，6. を１０回繰り返して，識別率の平均値と標準偏差を算出せよ．

In [121]:
knn=neighbors.KNeighborsClassifier(n_neighbors=1)
dataset=['yXT_car.csv','yXT_seeds.csv','yXT_yeast.csv','yXT_winequality-white.csv']
score=[]
mean=[]
std=[]
for i in range(len(dataset)):
    data=pd.read_csv(dataset[i],header=None)
    for j in range(10):
        xtrain,xtest,ytrain,ytest=train_test_split(data, data[0], test_size=0.3, stratify=data[0])
        knn.fit(xtrain, ytrain)
        score.append(knn.score(xtest,ytest))
    mean.append(np.mean(score))
    std.append(np.std(score))
print("mean:%s \n std:%s" %(mean, std))

mean:[0.9996146435452793, 0.9974263693916873, 0.9982842462611249, 0.9039852935393811] 
 std:[0.0007707129094412402, 0.00561609912846056, 0.0047433059912571135, 0.16345174725595227]


8. k-最近傍識別器 の k=1,2,3,4,5 を試せ． ただし，重みつきの voting をせよ． 重みは exp( - (ユークリッド距離)^2/(2σ^2)) で求めよ． ただし，σは訓練用データの距離値の中間値とする． これは，距離行列 D をまず計算し，median(D(:)) から求める．

In [143]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from numpy import array as matrix, arange
import math

data=pd.read_csv("yXT_wine.csv", header=None)
xtrain,xtest,ytrain,ytest=train_test_split(data, data[0], test_size=0.3, stratify=data[0],random_state=1)

def compute_distance_1(X):
  # determin dimensions of data matrix
    m,n = X.shape
  # initialize squared EDM D
    D = np.zeros([n, n])
  # iterate over upper triangle of D
    for i in range(n):
        for j in range(i+1, n):
            d = X[:,i] - X[:,j]
            D[i,j] = np.dot(d, d)**(1/2)
            D[j,i] = D[i,j]
    return D
D1=compute_distance_1(matrix(xtrain.iloc[:,1:].transpose()))

def compute_distance_2(X):
  # determin dimensions of data matrix
    m,n = X.shape
  # initialize squared EDM D
    D = np.zeros([n, n])
  # iterate over upper triangle of D
    for i in range(n):
        for j in range(i+1, n):
            D[i,j] = la.norm(X[:, i] - X[:, j])
            D[j,i] = D[i,j]    #*1
    return D
D2=compute_distance_2(matrix(xtrain.iloc[:,1:].transpose()))
sigma=np.median(D2)

def gaussian_weight(dist, sigma = sigma):
    """ Input a distance and return it`s weight"""
    weight = np.exp(-dist**2/(2*sigma**2))
    return weight
 
### weighted_KNN
def weighted_classify(input, dataSet, label, k):
    classification=[]
    for i in range(input.shape[0]):
        
        dataSize = dataSet.shape[0]
        diff = np.tile(input.iloc[i,1::], (dataSize, 1)) - dataSet.iloc[:,1:]
        sqdiff=diff**2
        squareDist = np.array([sum(sqdiff.iloc[x,:]) for x in range(124)])
        dist = squareDist**0.5
        #print(input, dist[0], dist[])
        sortedDistIndex = np.argsort(dist)
 
        classCount = {}
        for i in range(k):
            index = sortedDistIndex[i]
            label=list(label)
            voteLabel = label[index]
            weight = gaussian_weight(dist[index]) 
        #print(index, dist[index],weight)
        ## 这里不再是加一，而是权重*1
            classCount[voteLabel] = classCount.get(voteLabel, 0) + weight*1
        
        #print(classCount)
        maxCount = 0
        
        for key, value in classCount.items():
            if value > maxCount:
                maxCount = value
                classes = key
        classification.append(classes)
    return classification
#predict=[]
for k in range(1,6):
    predict=weighted_classify(xtest, xtrain,ytrain,k)
    print("k=%s:",%(k), predict)

SyntaxError: invalid syntax (<ipython-input-143-5d972c27f4f4>, line 80)

In [140]:
xtest.iloc[0,1::]

1      13.52
2       3.17
3       2.72
4      23.50
5      97.00
6       1.55
7       0.52
8       0.50
9       0.55
10      4.35
11      0.89
12      2.06
13    520.00
Name: 142, dtype: float64

In [131]:
def gaussian(dist, sigma = 285.0):
    """ Input a distance and return it`s weight"""
    weight = np.exp(-dist**2/(2*sigma**2))
    return weight
 
### 加权KNN
def weighted_classify(input, dataSet, label, k):
    for i in range(input.shape[0]):
        
        dataSize = dataSet.shape[0]
        diff = np.tile(input.iloc[i,1::], (dataSize, 1)) - dataSet.iloc[:,1:]
        sqdiff=diff**2
        squareDist = np.array([sum(sqdiff.iloc[x,:]) for x in range(124)])
        dist = squareDist**0.5
    #print(input, dist[0], dist[1164])
        sortedDistIndex = np.argsort(dist)
 
        classCount = {}
        for i in range(k):
            index = sortedDistIndex[i]
            label=list(label)
            voteLabel = label[index]
            weight = gaussian(dist[index]) 
        #print(index, dist[index],weight)
        ## 这里不再是加一，而是权重*1
            classCount[voteLabel] = classCount.get(voteLabel, 0) + weight*1
        
        maxCount = 0
    #print(classCount)
   
        for key, value in classCount.items():
            if value > maxCount:
                maxCount = value
                classes = key
        
    return classes
pre=weighted_classify(xtest, xtrain,ytrain,2)
pre

3

In [89]:
diff = np.tile(xtest.iloc[1,1::], (124, 1)) - xtrain.iloc[:,1:]
diff=diff**2
dist=np.array([sum(diff.iloc[x,:]) for x in range(124)])
#np.array([sum(diff.iloc[1,::]))
np.argsort(dist)[5]
list(ytrain)[25]
classCount = {3: 0.8626847069660394, 2: 0.7136690068267258}
classCount.items()
for key, value in classCount.items():
    print(key)
    print(value)

3
0.8626847069660394
2
0.7136690068267258


515.067039034726

124